In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('student.csv')

In [3]:
df.head(2)

,Unnamed: 0,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,Total Score,Average
0,0,female,group B,bachelor's degree,standard,none,72,72,74,218,72.666667
1,1,female,group C,some college,standard,completed,69,90,88,247,82.333333


## Starting the Model Part

In [4]:
## Independent Feature
x=df.drop(['math score'], axis=1)
y=df['math score'] ## Output or Dependent Feature

In [5]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   1000 non-null   int64  
 1   gender                       1000 non-null   object 
 2   race/ethnicity               1000 non-null   object 
 3   parental level of education  1000 non-null   object 
 4   lunch                        1000 non-null   object 
 5   test preparation course      1000 non-null   object 
 6   reading score                1000 non-null   int64  
 7   writing score                1000 non-null   int64  
 8   Total Score                  1000 non-null   int64  
 9   Average                      1000 non-null   float64
dtypes: float64(1), int64(4), object(5)
memory usage: 78.3+ KB


## Creating the num feature from x

In [6]:
num_feature=[feature for feature in x.columns if df[feature].dtype !='object']
cat_feature=[feature for feature in x.columns if df[feature].dtype == 'object']

In [7]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

In [8]:
num_transfer=StandardScaler()
one_transfer=OneHotEncoder()
preprocessor = ColumnTransformer(
    [
        ('Onehotencoding',one_transfer,cat_feature),
        ("StandardScaler",num_transfer,num_feature)
    ]
)

In [9]:
x=preprocessor.fit_transform(x)

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)

In [11]:
x_train.shape, x_test.shape , y_train.shape , y_test.shape

((800, 22), (200, 22), (800,), (200,))

In [12]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [13]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import AdaBoostRegressor
model_list=[]
r2_list=[]

## Now we have to use different different model from which we get the best accuracy we will use it

In [14]:
models={
    ('Linearregressor',LinearRegression()),
    ('Lasso',Lasso()),
    ('Ridge',Ridge()),
    ('KNeighborsRegressor',KNeighborsRegressor()),
    ('DecisionTreeRegressor',DecisionTreeRegressor()),
    ('RandomForestRegressor',RandomForestRegressor()),
    # 'CatBoostingRegressor',CatBoostRegressor(verbose=False),
    ('AdaBoostingRegressor',AdaBoostRegressor())
}
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [15]:
for name,model in models:
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    print("")
    print(model)
    print(r2_score(y_test,y_pred))
    print(mean_absolute_error(y_test,y_pred))
    print(mean_squared_error(y_test,y_pred))
    model_list.append(model)
    r2_list.append(r2_score(y_test,y_pred))


LinearRegression()
1.0
1.787014980436652e-14
4.919967693682406e-28

Ridge()
0.9994954966174101
0.2751457242026783
0.11817778584488398



KNeighborsRegressor()
0.8780793378237025
4.189
28.559399999999997

RandomForestRegressor()
0.9588218015885238
2.3887
9.645819

Lasso()
0.8942910098119751
3.967000616549585
24.7618843310613

AdaBoostRegressor()
0.9182368398370921
3.487126530250106
19.15267481880949

DecisionTreeRegressor()
0.9208310160556792
3.145
18.545


In [16]:
new_df=pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model_Name','R2_Score']).sort_values(by=['R2_Score'],ascending=False)

In [17]:
regg=LinearRegression()
regg.fit(x_train,y_train)
y_pred2=regg.predict(x_test)
print(r2_score(y_test,y_pred2),mean_squared_error(y_test,y_pred2),mean_absolute_error(y_test,y_pred2))

1.0 4.919967693682406e-28 1.787014980436652e-14


In [18]:
ped_df=pd.DataFrame({"Actual Value":y_test,"Predicted Value": y_pred,"Difference" : y_test-y_pred})

In [19]:
ped_df

,Actual Value,Predicted Value,Difference
540,75,73.0,2.0
537,51,58.0,-7.0
663,65,55.0,10.0
920,69,68.0,1.0
606,85,80.0,5.0
...,...,...,...
488,64,65.0,-1.0
347,77,82.0,-5.0
246,89,87.0,2.0
874,43,44.0,-1.0
